<a href="https://colab.research.google.com/github/boknilev/lm-intervention/blob/master/lm_intervention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pytorch-transformers
#!pip install spacy ftfy==4.4.3
#!python -m spacy download en
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

from pytorch_transformers import GPT2Tokenizer

In [2]:
from experiment import Intervention, Model

In [3]:
DEVICE = 'cuda'

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = Model(DEVICE)

In [5]:
# Currently not implemented/needed functions. 
def plot_log_probs(layer_to_candidate1_log_probs, layer_to_candidate2_log_probs):
    
    raise NotImplementedError
        
def print_neuron_hook(module, input, output, position, neuron):
        #print(output.shape) 
        print(output[0][position][neuron])
        
def print_all_hook(module, input, output, position):
    #print(output.shape) 
    print(output[0][position])

### Bookkeeping for all the experiments

ToDo: actually run all of them. 


In [6]:
'''
Test experiment
'''
base_sentence = "The {} said that"
biased_word = "teacher"
intervention = Intervention(
        tokenizer,
        base_sentence,
        [biased_word, "man", "woman"],
        ["he", "she"],
        device=DEVICE)

In [7]:
# Potential other sentences we could check. They could amplify the effect
# To Do: make different interventions with each after finalizing list.
base_sentence = "The {} said that"
base_sentence = "The {} yelled that"
base_sentence = "The {} whispered that"
base_sentence = "The {} wanted that"
base_sentence = "The {} desired that"
base_sentence = "The {} wished that"

base_sentence = "A {} caught the ball."
base_sentence = "A {} cooked the dinner."
base_sentence = "The {} baked the cake."
base_sentence = "A {} raised the gun."

In [8]:
# note: does not include teacher
profession_interventions = []
with open('professions.json', 'r') as f:
    for l in f: 
        # there is only one line that eval's to an array
        for j in eval(l):
            biased_word = j[0]
            profession_interventions.append(
                Intervention(
                    tokenizer,
                    "The {} said that",
                    [biased_word, "man", "woman"],
                    ["he", "she"]))

In [9]:
# ToDo: Need to double check that the sentences all make sense
male_interventions = []
with open('male_word.txt', 'r') as f:
    for l in f:
        # Strip off the \n
        biased_word = l[:-1]
        male_interventions.append(
                Intervention(
                    tokenizer,
                    "The {} said that",
                    [biased_word, "man", "woman"],
                    ["he", "she"]))

In [10]:
# ToDo: Need to double check that the sentences all make sense
female_interventions = []
with open('female_word.txt', 'r') as f:
    for l in f:
        # Strip off the \n
        biased_word = l[:-1]
        female_interventions.append(
                Intervention(
                    tokenizer,
                    "The {} said that",
                    [biased_word, "man", "woman"],
                    ["he", "she"]))

## One Test Experiment Run

ToDo: add eval/vis of result

In [12]:

""" Code draws on https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_gpt2.py """        
c1, c1_probs, c2, c2_probs = model.neuron_intervention_single_experiment(intervention)
# TODO: we need to look at the log prob distribution over the candidates and how that changes by intervention  


  0%|          | 0/12 [00:00<?, ?it/s]

Base case: The teacher said that ____
Ġhe: 9.81%
Ġshe: 12.17%



100%|██████████| 12/12 [01:29<00:00,  7.45s/it]

In [13]:
def convert_result_to_pd(intervention, c1_probs, c2_probs):
    results = []
    for c1_neurons, c2_neurons in zip(c1_probs.items(), c2_probs.items()):
        cur_layer = c1_neurons[0]
        first_probs = c1_neurons[1]
        second_probs = c2_neurons[1]
        for ix, (p1, p2) in enumerate(zip(first_probs, second_probs)):
            results.append({
                 'base_string': intervention.base_strings[0],
                 'first_condition': intervention.candidates[0],
                 'second_condition': intervention.candidates[1],
                 'p1': float(p1),
                 'p2': float(p2),
                 'layer': cur_layer,
                 'neuron': ix})
    return pd.DataFrame(results)
df = convert_result_to_pd(intervention, c1_probs, c2_probs)
df.head()

,base_string,first_condition,second_condition,p1,p2,layer,neuron
0,The teacher said that,Ġhe,Ġshe,0.036626,0.014896,0,0
1,The teacher said that,Ġhe,Ġshe,0.035116,0.001850,0,1
2,The teacher said that,Ġhe,Ġshe,0.078670,0.021347,0,2
3,The teacher said that,Ġhe,Ġshe,0.036290,0.020767,0,3
4,The teacher said that,Ġhe,Ġshe,0.101922,0.072067,0,4


In [14]:
'''
Report aggregate
'''
print('more probable candidate per layer, across all neurons in the layer')
print('candidate1:', intervention.candidates[0], c1)
print('candidate2:', intervention.candidates[1], c2)

more probable candidate per layer, across all neurons in the layer
candidate1: Ġhe Counter({0: 672, 2: 668, 1: 664, 3: 633, 4: 569, 5: 561, 6: 428, 7: 375, 8: 359, 9: 158, 10: 37})
candidate2: Ġshe Counter({11: 768, 10: 731, 9: 610, 8: 409, 7: 393, 6: 340, 5: 207, 4: 199, 3: 135, 1: 104, 2: 100, 0: 96})
